<a href="https://colab.research.google.com/github/MohamedElgharbawy/Z3Performance/blob/main/Z3_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Plotting Z3 Performance

This program plots Z3's performance when solving theorems with n random variables and a random set of [1, n<sup>2</sup>] constraints.

In [1]:
from z3 import *
import random
import operator
import time
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import os
import errno
import stopit

In [3]:
# Utils
ops = [operator.lt, operator.le, operator.ge, operator.gt]
twenty_min = 1200 # seconds

def write_to_dir(filename, contents):
  """
  Creates directory if it doesn't exist, deletes its contents, then writes to it
  """
  if not os.path.exists(os.path.dirname(filename)):
    try:
        os.makedirs(os.path.dirname(filename))
    except OSError as exc:
        if exc.errno != errno.EEXIST:
          raise

  for f in os.listdir(filename):
    os.remove(os.path.join(filename, f))

  with open(filename, "w") as f:
    f.write(contents)

def try_check_model(solver, timeout):
  """ 
  input: solver, timeout (seconds)
  output: satisfiability (sat/unsat)
  throws: TimeoutError
  """
  with stopit.SignalTimeout(timeout) as to_ctx_mgr:
    satisfiability = solver.check()

  if to_ctx_mgr.state != to_ctx_mgr.EXECUTED:
    raise TimeoutError(f"Solver timed out after {timeout} seconds.")

  return str(satisfiability)
    

In [4]:
def z3_performance(num_var: int, use_max: bool, timeout: int, inc_constraints=1):
  """
  Measure Z3's performance to check a model for num_var variables with
  1 - num_var**2 constraints.
  Returns the a tuple, a list of the number of constraints, and a list of the
  corresponding times.
  """
  X = IntVector('x', num_var)

  if use_max:
    solver = Optimize()
    solver.maximize(Sum(X))
  else:
    solver = Solver()

  # Graphing lists
  num_constraints = []
  times = []
  num_unsat = 0

  # Constraints
  constraints = []
  for i in range(1, num_var**2 + 1, inc_constraints):
    print(f"{num_var}: {i} constraints")
    num_constraints.append(i)

    # Get a random operator
    op = random.choice(ops)
    
    # Create a random constraint
    constraint = op(sum(random.randint(-10, 10)*X[j] for j in range(0,num_var)), random.randint(-10, 10))
    constraints.append(constraint)
    solver.add(constraint)

    # Write problem to smt lib txt file
    filename = f"problems/{num_var}/{num_var},num_constraints={i}.txt"
    write_to_dir(filename, solver.sexpr())

    # Check model
    start_time = time.monotonic()
    satisfiability = None
    try:
      satisfiability = try_check_model(solver, timeout)
      final_time = time.monotonic() - start_time
      times.append(round(final_time, 5))
    except:
      times.append(np.nan)

    filename = f"models/{num_var}/{num_var},num_constraints={i}.txt"
    if satisfiability == "unsat":
      num_unsat += 1
      write_to_dir(filename, "unsat")
    elif satisfiability == "sat":
      write_to_dir(filename, solver.model().sexpr())
    else:
      write_to_dir(filename, "timeout")
    
    if num_unsat == 10:
      break

  return num_constraints, times

In [10]:
def plot_one_z3_performance(num_var: int, use_max: bool, timeout: int, inc_constraints=1):
  """
  Measure and plot Z3's performance for num_var variables
  """
  num_constraints, times = z3_performance(num_var, use_max, timeout, inc_constraints)
  
  for x, y in zip(num_constraints, times):
    if np.isnan(y):
      plt.text(x, timeout, "x", fontsize=12, color="red", ha="center", va="center")
      plt.legend()

  # Replace nan times with timeout before plotting
  times = np.nan_to_num(times, nan=timeout)
  plt.plot(num_constraints, times)

  plt.title(f"Z3 Performance for {num_var} variables")
  plt.xlabel("Number of constraints")
  plt.ylabel("Time (seconds)")
  plt.savefig(f"plots/{num_var},use_max={use_max}")
  plt.clf()

In [15]:
random.seed(10)
plot_one_z3_performance(10, use_max=True, timeout=1)

10: 1 constraints
10: 2 constraints
10: 3 constraints
10: 4 constraints
10: 5 constraints
10: 6 constraints
10: 7 constraints
10: 8 constraints
10: 9 constraints
10: 10 constraints
10: 11 constraints
10: 12 constraints
10: 13 constraints
10: 14 constraints
